In [ ]:
import numpy as np
import pandas as pd
import gzip

# Genome sequence

In [ ]:
# bedtools getfasta -fi hg19.fa -bed test.bed -name > test.fa

In [ ]:
# seq2mat
acgt2num = {'A': 0, 'C': 1, 'G': 2, 'T': 3}
def base2num(seq, l=1000):
    h = 4
    l = len(seq)
    mat = np.zeros((h, l), dtype=int)
    for i in range(l):
        if seq[i] != 'N':
            mat[acgt2num[seq[i]], i] = 1
    return mat.reshape((h, -1))

In [ ]:
with open('test.fa', 'r') as f1:
    fa1 = f1.readlines()
ss = []
for i in range(len(fa1)//2):
    ss.append(fa1[2*i+1])
ss_mats = []
for i in range(len(ss)):
    ss_mat = base2num(ss[i].strip().upper())
    ss_mats.append(ss_mat)
ss_mats = np.array(ss_mats, dtype='uint8')
np.save('test_seqs.npy', ss_mats)

# Openness

To incorporate the information of chromatin accessibility, we adopted OpenAnnotate (http://159.226.47.242:65533/openness/anno/) to efficiently calculate the raw read open scores of CREs per base pair. We then derived the chromatin open score per base pair by averaging the raw read open scores across replicates for each respective cell type.

In [ ]:
l = 1000 #Length of sequence, eg. 1000bp
replicates = 5 #Number of replicates
with gzip.open('readopen.txt.gz' % (cell, cre), 'r') as f:
    head = f.readlines()
opens = []
for i in range(len(head)):
    opens.append(str(head[i], encoding = "utf-8").strip().split('\t')[4:])
openness = np.array(opens, dtype=np.float).reshape(len(opens)//l, l, replicates).transpose((0,2,1))
openness2 = np.log10(np.mean(openness, axis=1, keepdims=True) + 1)
np.save('test_opens.npy', openness2)

# HiChIP

To incorporate the information of chromatin interaction, we first calculated the number of chromatin loops per base pair for each CRE, and then obtained the chromatin loop score after logarithmic transformation.

In [ ]:
l = 1000 #Length of sequence, eg. 1000bp
eps = 1e-8
with open('test_loop.bed', 'r') as f:
    head = f.readlines()
hcs = []
for i in range(len(head)):
    hcs.append(str(head[i]).strip().split('\t')[-1])
hichip = np.array(hcs, dtype='uint8').reshape(len(hcs)//l, 1, l)
hichip2 = np.log10(hichip + 1 + eps)
np.save('test_loops.npy', hichip2)